This is the prototype of the Mean Field

There will be three main modules:

- 1) Agent

- 2) MeanField

- 3) BlackBoard

In [6]:
import numpy as np
import scipy as sp
import ode
from sliding_window import *

### Agent

In [7]:
class Agent:
    
    def L_l(self, q, q_dot, u):
        return
    
    def L_l_q_dot(self, t, q, q_dot, u, **kwargs):
        return 
    
    def H_l_nou(self, q, p, lambda_l):
        return 

    def qp_rhs_l_nou(self, t, qp_vec, **kwargs):
        dim = len(qp_vec)/4
        q = qp_vec[:dim]
        p = qp_vec[dim:2*dim]
        q_D = qp_vec[2*dim:3*dim]
        p_D = qp_vec[3*dim:]
        u = kwargs['u_0']
        # for q-dot
        q_dot =  np.zeros(np.shape(p))
        # for p-dot
        p_dot =  np.zeros(np.shape(q))
        q_D_dot =  np.zeros(np.shape(p))
        p_D_dot =  np.zeros(np.shape(q))
        return np.concatenate([q_dot, p_dot, q_D_dot, p_D_dot])
       
    def u_rhs_l_nou(self, t, u_vec, **kwargs):
        qp_vec = kwargs['qp_vec']
        dim = len(qp_vec)/4
        q = qp_vec[:dim]
        p = qp_vec[dim:2*dim]
        q_D = qp_vec[2*dim:3*dim]
        p_D = qp_vec[3*dim:]
        Gamma = kwargs['Gamma']
        # for u-dot
        return -1*Gamma*np.zeros(np.shape(u_vec))

    def H_l_u(self, q, p, u):
        return 
    
    def qp_rhs_l_u(self, t, qp_vec, **kwargs):
        dim = len(qp_vec)/4
        q = qp_vec[:dim]
        p = qp_vec[dim:2*dim]
        q_D = qp_vec[2*dim:3*dim]
        p_D = qp_vec[3*dim:]
        u = kwargs['u_0']
        # for q-dot
        q_dot =  np.zeros(np.shape(p))
        # for p-dot
        p_dot =  np.zeros(np.shape(q))
        q_D_dot =  np.zeros(np.shape(p))
        p_D_dot =  np.zeros(np.shape(q))
        return np.concatenate([q_dot, p_dot, q_D_dot, p_D_dot])
       
    def u_rhs_l_u(self, t, u_vec, **kwargs):
        qp_vec = kwargs['qp_vec']
        dim = len(qp_vec)/4
        q = qp_vec[:dim]
        p = qp_vec[dim:2*dim]
        q_D = qp_vec[2*dim:3*dim]
        p_D = qp_vec[3*dim:]
        Gamma = kwargs['Gamma']
        # for u-dot
        return -1*Gamma*np.zeros(np.shape(u_vec))

    def qp_rhs_H_s(self, t, qp_vec, **kwargs):
        # This will include a term for qp_rhs_l and a term for qp_rhs_MF: 
        qp_vec
        qp_rhs_H_s = (1-alpha)*qp_rhs_l(self, qp_vec)+alpha*qp_rhs_MF()
        return np.concatenate([q_dot, p_dot, q_D_dot, p_D_dot])
       
    def u_rhs_H_s(self, t, u_vec, **kwargs):
        return np.concatenate([q_dot, p_dot, q_D_dot, p_D_dot])
        Gamma = kwargs['Gamma']
        # for u-dot
        return -1*Gamma*np.zeros(np.shape(u_vec))

    def H_l_D():
        pass
        
    def L_l_D():
        pass
        
    def L_l_D_q_Dot():
        pass
    # Inputs for numerical propagator
    q_0 = np.array([0])
    p_0 = np.array([0])
    q_D = np.array([0])
    p_D = np.array([0])
    u_0 = np.array([0])
    qpu_vec = np.hstack([q_0, p_0, q_D, p_D, u_0])
    state_dim = 1 
    Gamma = 1 
       
    # Inputs for numerical integration
    integrateTol = 10**-3
    integrateMaxIter = 40
       
    # Inputs for sliding window
    t_0 = 0 
    T =  2
    K=1 
    t_terminal = 2 
    n_s = 10



### MeanField

In [8]:
class MeanField:
    
    def __init__(self):
        pass
    
    def H_MF_nou(self, q, p):
        pass
        
    def qp_rhs_H_MF_nou(self, t, qp_vec, **kwargs):
        pass

    def H_MF_u(self, q, p):
        pass
        
    def qp_rhs_H_MF_u(self, t, qp_vec, **kwargs):
        pass
          

IndentationError: expected an indented block (<ipython-input-8-7f3eedecd191>, line 5)